#### when processing big bench task datasets --> notice that size of unique indeces (idx field) != size of task dataset

In [5]:
import pandas as pd
from datasets import load_dataset
idx_df = pd.read_csv("./indx_bigbench.csv")

idx_mismatch = idx_df[(idx_df['task_size'] != idx_df['unique_indeces'])]
print(f"number mismatches: {len(idx_mismatch)}, total tasks: {len(idx_df)}")

/Users/sunil/opt/anaconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


number mismatches: 24, total tasks: 96


In [3]:
idx_mismatch.head()

,Unnamed: 0,task,task_size,unique_indeces
0,0,cs_algorithms,1056,800
1,1,bbq_lite_json,12866,1600
5,5,hhh_alignment,179,49
10,10,minute_mysteries_qa,383,220
16,16,intersect_geometry,50000,30473


In [6]:
# check for duplicates across all mismatch task datasets
duplicates = []
for task in idx_mismatch['task']:
    mismatch_ds = load_dataset("tasksource/bigbench", 'cs_algorithms', split='train') # tb
    u_inputs = len(mismatch_ds.unique('inputs'))
    if u_inputs != len(mismatch_ds):
        print(task)
        duplicates.append(task)

Found cached dataset bigbench (/Users/sunil/.cache/huggingface/datasets/tasksource___bigbench/cs_algorithms/1.0.0/d9f074b582e19cb8b59cbcd48e43c1c15cf316f7c774c526da2bca369f786cc3)
Found cached dataset bigbench (/Users/sunil/.cache/huggingface/datasets/tasksource___bigbench/cs_algorithms/1.0.0/d9f074b582e19cb8b59cbcd48e43c1c15cf316f7c774c526da2bca369f786cc3)
Found cached dataset bigbench (/Users/sunil/.cache/huggingface/datasets/tasksource___bigbench/cs_algorithms/1.0.0/d9f074b582e19cb8b59cbcd48e43c1c15cf316f7c774c526da2bca369f786cc3)
Found cached dataset bigbench (/Users/sunil/.cache/huggingface/datasets/tasksource___bigbench/cs_algorithms/1.0.0/d9f074b582e19cb8b59cbcd48e43c1c15cf316f7c774c526da2bca369f786cc3)
Found cached dataset bigbench (/Users/sunil/.cache/huggingface/datasets/tasksource___bigbench/cs_algorithms/1.0.0/d9f074b582e19cb8b59cbcd48e43c1c15cf316f7c774c526da2bca369f786cc3)
Found cached dataset bigbench (/Users/sunil/.cache/huggingface/datasets/tasksource___bigbench/cs_alg

In [7]:
print(f"amount of duplicate questions across tasks with mismatch indeces: {len(duplicates)}")

amount of duplicate questions across tasks with mismatch indeces: 0


In [19]:
# example mismatch data
mismatch_ex_tb = load_dataset("tasksource/bigbench", 'cs_algorithms', split='train') # tb

Found cached dataset bigbench (/Users/sunil/.cache/huggingface/datasets/tasksource___bigbench/cs_algorithms/1.0.0/d9f074b582e19cb8b59cbcd48e43c1c15cf316f7c774c526da2bca369f786cc3)


In [20]:
# example
print(f"unique idx: {len(mismatch_ex_tb.unique('idx'))}, unique inputs: {len(mismatch_ex_tb.unique('inputs'))}, size of task: {len(mismatch_ex_tb)}")

unique idx: 800, unique inputs: 1056, size of task: 1056


In [21]:
# example mismatch
df = mismatch_ex_tb.to_pandas()
df[df['idx'] == 0]

,inputs,targets,multiple_choice_targets,multiple_choice_scores,idx
0,"Given two strings, determine the length of the...",[8],"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",0
256,Determine whether the given sequence of parent...,[Invalid],"[Valid, Invalid]","[0, 1]",0


### conclusion: idx not unique; same idx does not imply same question
#### solution -> use row number as true idx
#### want to query the original row from true idx: load dataset, add column: `true_idx = [i for i in range(len(task_dataset))]`
#### query using `true_idx`